# Prepare & Concept

## Augmentation
- 컴퓨터 과학 분야에서 한정된 예산 안에서 최대한 많은 데이터를 확보하기 위해 데이터를 부풀리는 작업을 augmentation 이라고 부른다.
- AI를 학습시키기 위한 기초 작업으로 쓰인다.

In [ ]:
import urllib.request

py_file_path = "[Part 2] 컴퓨터 자동화 기초/[Chapter 5] 컴퓨터! 디자인 작업도 자신 있지 1분 안에 모두 처리해!/2_5_6_예제를 위해 증명사진 1,000장이 필요해졌네요/sample_picture.png"
"https://raw.githubusercontent.com/" + "needleworm/bhban_rpa/master/" + urllib.request.pathname2url(py_file_path)

In [ ]:
# download sample_picture
!wget https://raw.githubusercontent.com/needleworm/bhban_rpa/master/%5BPart%202%5D%20%EC%BB%B4%ED%93%A8%ED%84%B0%20%EC%9E%90%EB%8F%99%ED%99%94%20%EA%B8%B0%EC%B4%88/%5BChapter%205%5D%20%EC%BB%B4%ED%93%A8%ED%84%B0%21%20%EB%94%94%EC%9E%90%EC%9D%B8%20%EC%9E%91%EC%97%85%EB%8F%84%20%EC%9E%90%EC%8B%A0%20%EC%9E%88%EC%A7%80%201%EB%B6%84%20%EC%95%88%EC%97%90%20%EB%AA%A8%EB%91%90%20%EC%B2%98%EB%A6%AC%ED%95%B4%21/2_5_6_%EC%98%88%EC%A0%9C%EB%A5%BC%20%EC%9C%84%ED%95%B4%20%EC%A6%9D%EB%AA%85%EC%82%AC%EC%A7%84%201%2C000%EC%9E%A5%EC%9D%B4%20%ED%95%84%EC%9A%94%ED%95%B4%EC%A1%8C%EB%84%A4%EC%9A%94/sample_picture.png

# RPA design code
- 목표 정하기
  1. 사진 한 장을 최대한 여러 장으로 부풀리기
  1. 그 중 동일한 사진은 단 한 장도 없어야 한다.

- 목표를 달성하는 데 필요한 작업 쪼개기
  1. 이미지를 좌우 대칭으로 놓는다. (X2)
  1. 이미지를 상하 대칭으로 놓는다. (X2)
  1. 이미지를 흑백 버전으로 생성한다. (X2)
  1. 이미지를 1도씩 회전시킨다. (X179)

#### 1. 이미지를 좌우/상하 대칭한다.
- ```transpose()``` 함수를 사용한다.
```python
>>> my_image.transpose(Image.FLIP_LEFT_RIGHT)
>>> my_image.transpose(Image.FLIP_TOP_BOTTOM)
```

#### 2. 이미지를 흑백 버전으로 만든다.
- ```convert()``` 함수를 사용한다.
```python
>>> my_image.convert("1")
```

#### 3. 이미지를 1도씩 회전시킨다.
- ```rotate()``` 함수를 사용한다. 회전시킨 후, 이미지 크기를 유지하기 위해 ```resize()``` 함수를 실행한다.
```python
>>> my_image.rotate(10)
>>> Xdim, Ydim = my_image.size()
>>> my_image = my_image.rotate(90)
>>> my_image = my_image.resize((Xdim, Ydim))
```

In [ ]:
%%writefile augmentation.py
#!usr/bin/bash python
#-*-coding:utf-8

import time
import os
from PIL import Image
import sys
import shutil

print("Process Start.")
start_time = time.time()

image_filename = sys.argv[1]

out_dir = "augmentation"
if out_dir not in os.listdir():
    os.mkdir(out_dir)
else:
    shutil.rmtree(out_dir)
    os.mkdir(out_dir)

image = Image.open(image_filename)
# note: using unpacking, instance
Xdim, Ydim = image.size

# init
COUNT = 1

# revised: f-formatting
# save copy of image
temp_new_file_name = f"{COUNT:05}.png"
COUNT += 1
image.save(out_dir + "/" + temp_new_file_name)
image.close()

FILELIST = [temp_new_file_name]



# Process 1: FLIP_LEFT_RIGHT
for i in range(len(FILELIST)):
    # import
    image = Image.open(out_dir + "/" + FILELIST[i])
    new_temp_name = f"{COUNT:05}.png"

    COUNT += 1
    
    # reverse
    image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # save
    image.save(out_dir + "/" + new_temp_name)
    image.close()

    FILELIST.append(new_temp_name)

# Proecss 2 : FLIP_TOP_BOTTOM
for i in range(len(FILELIST)):
    # import
    image = Image.open(out_dir + "/" + FILELIST[i])
    new_temp_name = f"{COUNT:05}.png"

    COUNT += 1

    # reverse
    image = image.transpose(Image.FLIP_TOP_BOTTOM)

    # save
    image.save(out_dir + "/" + new_temp_name)
    image.close()

    FILELIST.append(new_temp_name)

# Process 3 : black & white
for i in range(len(FILELIST)):
    # import
    image = Image.open(out_dir + "/" + FILELIST[i])
    new_temp_name = f"{COUNT:05}.png"

    COUNT += 1

    # black & white
    image = image.convert("1")

    # save
    image.save(out_dir + "/" + new_temp_name)
    image.close()

    FILELIST.append(new_temp_name)

# Process 4 : rotation
for el in FILELIST:
    for i in range(180):
        # create 1000 pieces from the original
        if COUNT > 10000:
            break
        
        # import
        image = Image.open(out_dir + "/" + el)
        new_temp_name = f"{COUNT:05}.png"

        COUNT += 1

        # rotate & resize
        image = image.rotate(i+1)
        image = image.resize((Xdim, Ydim))

        # save
        image.save(out_dir + "/" + new_temp_name)
        image.close()

print("Process Done.")
end_time = time.time()
print("Peter's " + str(end_time - start_time) + " seconds.")

Overwriting augmentation.py


In [ ]:
%run augmentation.py sample_picture.png

Process Start.
Process Done.
Peter's 19.469033002853394 seconds.


# Exercise

In [ ]:
cnt = 1

# zfill
f"{cnt:05}"

'00001'